In [15]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
df_train = pd.read_csv(r'logs\inference\train_features.csv')
df_valid = pd.read_csv(r'logs\inference\valid_features.csv')

In [3]:
train = df_train.iloc[:,1:].values
valid = df_valid.iloc[:,1:].values

In [4]:
train.shape

(450, 512)

In [5]:
train_unit = np.array([vec/np.linalg.norm(vec) for vec in train])
mean_vector = np.mean(train_unit,axis=0)
mean_vector = mean_vector / np.linalg.norm(mean_vector)

In [6]:
def calc_anomalyDeg(x, mean_vector):
    x = x / np.linalg.norm(x)
    return 1 - np.dot(x,mean_vector)

In [7]:
train_anos = np.array([])
for t_vec in train_unit:
    train_anos = np.append(train_anos,calc_anomalyDeg(t_vec, mean_vector))

In [8]:
ano_m = np.mean(train_anos)
ano_S = np.mean(train_anos**2)

In [20]:
m_mo = (2*ano_m**2)/(ano_S - ano_m**2)
s_mo = (ano_S - ano_m**2)/(2 * ano_m)

In [21]:
m_mo, s_mo

(6.532391273691134, 0.001572364822163951)

In [72]:
th = stats.chi2.ppf(0.999,df=m_mo,scale=s_mo)
th

0.03688569052460508

In [77]:
valid_anos = np.array([])
for v_vec in valid:
    valid_anos = np.append(valid_anos, calc_anomalyDeg(v_vec, mean_vector))

In [78]:
df_valid_result = [1 if ano >= th else 0 for ano in valid_anos]

In [79]:
valid_path = df_valid.iloc[:,0].values

result = np.vstack([valid_path, valid_anos,df_valid_result])

df_result = pd.DataFrame(result).T

In [80]:
df_result.to_csv('logs/inference/result.csv')